<a href="https://www.kaggle.com/code/nurulsakinah/academic-success-classification-rf-lgbm?scriptVersionId=194502360" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import randint, uniform
from sklearn import metrics
from lightgbm import LGBMClassifier
from sklearn.metrics import make_scorer,log_loss
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm




# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load the dataset

In [ ]:
# read file
df = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv')


# Data Cleaning
check missing data

In [ ]:
# check missing values
df.isnull().sum()

# Exploratory Data Analysis
To get an overview of the data, we will check the unique values in each columns and the type of data


In [ ]:
# calculate unique values in each column
unique_col = df.nunique()
print(unique_col)

In [ ]:
# check type of data
print(df.info())

Create a list to store the continuous and categorical variables

In [ ]:
# get the categorical variables
categorical_col = df.select_dtypes(include='int').columns.tolist()
# exclude Age at enrollment as that would be continous
categorical_col.remove('Age at enrollment')
print('Categorical variables: ', categorical_col)
print(len(categorical_col))


# get continous variable
cont_col = df.select_dtypes(exclude=['int', 'object']).columns.tolist()
cont_col.append('Age at enrollment')
print('Continuous variables: ', cont_col)
print(len(cont_col))

## Data Visualisation
In this section, each columns will be explored and visualized to get an overview of the data distributions. <br>
The proportion of the 'Target' is plotted with countplot. <br>
'Graduate' has the highest % 

In [ ]:
# Categorical variable
# lets first check the categorical variable
# Create a count plot for the 'Target' column
plt.figure(figsize=(5, 3))
sns.countplot(x='Target', data=df)
plt.title('Count Plot of Target')
plt.xlabel('Target')
plt.ylabel('Count')
plt.show()
# the data is unbalanced

# Calculate percentage of target
percent_target = round((df['Target'].value_counts()/len(df)*100),2)
print(percent_target)

### Boxplots
Boxplot is used tovisualise the outliers and the mean of the continuous columns.<br>
The boxplots shows outliers exist in some of the variables. 

In [ ]:
# lets do boxplot to identify outliers
# Creating grid of subplots
fig, ax = plt.subplots(3, 3, figsize=(9, 9))

ax = ax.flatten()
# Loop through columns and plot box plots
for idx, col in enumerate(cont_col):
    sns.boxplot(data=df, y=col, ax=ax[idx])
    ax[idx].set_title(f'Box Plot of {col}', fontsize=10)
    ax[idx].set_ylabel(col)

# Hide the empty subplot
if len(cont_col) < len(ax):
    ax[len(cont_col)].set_visible(False)
    
# Adjust the layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

### Countplot and KDE plot
Create countplot for categorical variables and KDE plot for continuous variables.<br>

In [ ]:
# Creating grid of subplots for distribution
fig, axis = plt.subplots(12, 3, figsize=(20, 60))

# Loop through columns and axes
for cols, ax in zip(df.drop(columns=['id', 'Target']).columns, axis.ravel()):
    if cols in categorical_col:
        # Get counts of each category and target combination
        counts = df.groupby(
            [cols, 'Target']).size().unstack(fill_value=0)

        # Plot the stacked bar chart
        counts.plot(kind='bar', stacked=True, ax=ax,
                    color=sns.color_palette("pastel"))
        ax.set_ylabel('Count', fontsize=10)
        ax.set_title(f'Categorical variable of {cols}', fontsize=10)
    else:
        # Plot hist and density plot for continuous feature
        sns.kdeplot(data=df,
                    x=df[cols], hue='Target', ax=ax, fill=True)
        ax.set_ylabel('Density', fontsize=10)
        ax.set_title(cols, fontsize=10)

# Adjust the layout to prevent overlap
plt.subplots_adjust(hspace=0.4)

plt.show()

In [ ]:
def count_cat_percent(variable):
    # Calculate the percentage of 1s in column
    count_ones = df[variable].sum()
    percentage_ones = round((count_ones /len(df['Educational special needs'])) * 100,2)
    print(f'Percentage of 1 in column {variable} is :',percentage_ones,'%')
    
count_cat_percent('Educational special needs')
count_cat_percent('International')
count_cat_percent('Debtor')

Educational special needs column looks like it has small category of 1. <br>
Calculate the percentage of some of the columns with small proportion. <br>
We will drop the Educational special needs and International columns in later steps as these has very little values and does not provide much learning to the model

# Data Preprocessing
## Handling Outliers 
### Detect and Visualise the outliers
The outliers were calculated for the cont. columns & visualised with their respective upper and lower bound. <br>
The scatterplot of curricular units 1st sem(grade) & curricular units 1st sem(grade) shows 0 as outliers which is 'dropout' Target. Makes sense as if you have 0 grade, meaning you did not score well and grade is 0. <br>
Not going to remove these  outliers as it could be a good learning for models

In [ ]:
def outliers_analysis(df):
    # save the percentage of outliers in a df
    outliers_df = pd.DataFrame()

    # calculate outliers
    for i in (cont_col):
        # Calculate IQR upper and lower limit
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        # Identify outliers
        outliers = df[(df[i] < lower) | (df[i] > upper)]
        outliers_percent = round(len(outliers) / len(df) * 100, 2)
        # Save outlier indices
        outlier_indices = outliers.index.tolist()

        # Create a new DataFrame with the current results
        new_row = pd.DataFrame(
            {'Column': [i],
             'Outliers Percentage': [outliers_percent],
             'Upper limit': [upper],
             'Lower limit': [lower],
             'Outlier index': [outlier_indices]})

        # Append the new DataFrame to the existing DataFrame
        outliers_df = pd.concat([outliers_df, new_row], ignore_index=True)

        # Plot scatterplot of the outliers
        plt.figure(figsize=(6, 2))
        sns.scatterplot(x=df.index,
                        y=df[i], hue=df['Target'], edgecolor='none', s=10)
        plt.title(f'Scatter Plot of {i} vs Index', fontsize=10)
        plt.xlabel('Index')
        plt.ylabel(i)
        plt.legend(title='Target')
        # plot lower and upper bound
        plt.axhline(lower, color='red', linestyle='--')
        plt.axhline(upper, color='red', linestyle='--')
        plt.show()

    # Sorting by column 'Country'
    outliers_df.sort_values(
        by=['Outliers Percentage'], ascending=True, inplace=True)

    # Plot outliers percentage by colummn
    plt.figure(figsize=(6, 2))
    sns.barplot(data=outliers_df, x='Column',
                y='Outliers Percentage', palette='viridis')

    # Add title and labels
    plt.title('Percentage of Outliers per Column', fontsize=10)
    plt.xlabel('Column')
    plt.ylabel('Outliers in Percentage')

    # Rotate x-axis labels for better readability if needed
    plt.xticks(rotation=90)

    # Display the plot
    plt.tight_layout()
    plt.show()

    return outliers_df


outliers_df = outliers_analysis(df)

### Outliers removal
The outliers in column 'Previous qualification (grade)','Admission grade' is removed.

In [ ]:
# Remove outliers
def outliers_removal(df, outliers_data, selected_column):
    
    # Filter the DataFrame to include only the desired rows
    filtered_df = outliers_data[outliers_data['Column'].isin(selected_column)]
    
    # Combine the lists in the 'outlier index' column into a single set
    outlier_indices = set()
    for i in filtered_df['Outlier index']:
        outlier_indices.update(i)
        
    # Remove outliers from the original DataFrame
    df_cleaned = df.drop(index=outlier_indices)
    
    return df_cleaned

# specify which column with outliers to be removed
selected_column = ['Previous qualification (grade)',
                   'Admission grade']

# create a cleaned df
df_cleaned = outliers_removal(df, outliers_df, selected_column)

## Correlation Analysis
Create correlation matrix to see if if there is multicollinearity between the variables. 

In [ ]:
def correlation_analysis(df):
    ''' visualise the correlation matrix,
    and return the highest correlated pairs'''

    # Correlation matrix
    df_num = df.select_dtypes(include=[np.number])

    # Compute the correlation matrix
    corr = df_num.corr()

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(20, 15))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)

    # Draw the heatmap with the mask
    sns.heatmap(corr, annot=True, mask=mask, cmap=cmap,
                fmt='.2f', cbar=True, annot_kws={"size": 8})

    # unstack the matrix
    corr_unstacked = corr.unstack()

    # Convert the Series to a DataFrame and reset the index
    corr_df = pd.DataFrame(corr_unstacked).reset_index()
    corr_df.columns = ['Feature1', 'Feature2', 'Correlation']

    # Remove self-correlations (correlation of a feature with itself)
    corr_df = corr_df[corr_df['Feature1'] != corr_df['Feature2']]

    # Get the absolute values of the correlations
    corr_df['Correlation'] = corr_df['Correlation'].abs()

    # Sort the DataFrame by absolute correlation values in descending order
    corr_df = corr_df.sort_values(by='Correlation', ascending=False)

    corr_df['sorted_features'] = corr_df.apply(lambda row: tuple(
        sorted([row['Feature1'], row['Feature2']])), axis=1)
    corr_df = corr_df.drop_duplicates(subset=['sorted_features'])

    # Extract the top 10 related features
    top_related = corr_df.head(10)

    return top_related


# top_correlated = correlation_analysis(df)
top_correlated = correlation_analysis(df_cleaned)

print('Top related features: ', top_correlated)

# Extract feature pairs for visualization
top_pairs = top_correlated[['Feature1', 'Feature2']].values

    
    


In [ ]:
# Create the subplots grid
fig, ax = plt.subplots(5, 2, figsize=(15, 15))  # Adjusted figsize for better visibility
ax = ax.flatten()  # Flatten in case of multi-dimensional axes array

# Loop through each pair and create a scatter plot in the corresponding subplot
for i, pair in enumerate(top_pairs):
    feature1, feature2 = pair
    sns.scatterplot(data=df, x=feature1, y=feature2, ax=ax[i])
    ax[i].set_title(f'{feature1} vs {feature2}')


# Adjust the layout to prevent overlap
plt.tight_layout()

plt.show()

## Feature selection
Features to drop: 
Curricular units 1st sem(enrolled) <br>
Curricular units 1st sem (credited) <br> 
Curricular units 1st sem (approved) <br>
Nacionality - we can distinguish this by international features. <br>
These are the features that has linear relationship based on the graph. <br>
I've decided to keep the 2nd sem as it seems more reasonable to take the latest result compared to 1st semester




In [ ]:
columns_drop = ['Curricular units 1st sem (enrolled)',
                'Curricular units 1st sem (credited)',
                'Curricular units 1st sem (approved)',
                'Nacionality',
               'Educational special needs']
df_cleaned.drop(columns_drop, axis=1, inplace=True)

# Model Building
## Split the data
Data is split into 75/25 ratio for train and test

In [ ]:
# encode the target variable
df_cleaned['Target'] = df_cleaned['Target'].map(
    {'Graduate': 2, 'Enrolled': 1, 'Dropout': 0})

# separate the Target as y
y = df_cleaned['Target']
# get X by dropping column id and Target from df
x = df_cleaned.drop(columns=['id', 'Target'])

# Split to train and test set by 75/25
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=0)


## Random Forest Model
### Hyperparameter tuning
Hyperparameter tuning to get the best parameter. <br>
Notes: tuning was done with RandomizedSearchCV, it takes a while so i will comment the code and set the best parameter found to variable 'rf_model'. <br>

In [ ]:
# # Define the parameter grid
# param_dist = {
#     'n_estimators': randint(100, 1000),  # Number of trees in the forest
#     'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
#     'max_depth': randint(10, 100),  # Maximum number of levels in the tree
#     'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
#     'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
#     'bootstrap': [True, False]  # Method of selecting samples for training each tree
# }

# # Initialize RandomizedSearchCV
# rf_search = RandomizedSearchCV(
#     estimator=rf, 
#     param_distributions=param_dist, 
#     n_iter=100,  # Number of parameter settings that are sampled
#     cv=3,  # 3-fold cross validation
#     verbose=2, 
#     random_state=42,
#     n_jobs=-1  # Use all available cores
# )

# # Fit the random search model
# rf_search.fit(x_train, y_train)

# # Create a variable for the best model
# best_rf = rf_search.best_estimator_

rf_model = RandomForestClassifier(
    bootstrap= False, 
    max_depth= 84, 
    max_features= 'auto', 
    min_samples_leaf= 4, 
    min_samples_split= 5, 
    n_estimators= 493)

# Fit the model on the training data
rf_model.fit(x_train, y_train)

# Predict on the test data
y_pred_rf = rf_model.predict(x_test)

### Feature Importance

In [ ]:
# check feature importances
feature_importances = pd.Series(rf_model.feature_importances_, index=x_train.columns)
feature_importances.sort_values(ascending=True, inplace=True)
feature_importances.plot.barh(color='green')
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance", fontsize=10)
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)

# Show the plot
plt.show()

## LightGBM Classifier
### Hyperparameter tuning
I feel that the hyperparameter tuning here can be improved, but due to limited time, this will be used for now

In [ ]:
# lgbm_params = {
#                  'objective': 'multiclass',
#                  'tree_learner': 'feature', 
#                  'n_estimators': randint(100, 1000), 
#                  'num_leaves': randint(100, 1000),
#                  'max_depth': [10, 20, 30, 40, 50],
#                  'verbose': -1, 
#                  'random_state': 42 
# }

# # Create the LGBMClassifier for multi-class classification
# lgbm_model = lgbm.LGBMClassifier(objective="multiclass", num_class=3)

# # Perform RandomizedSearchCV with StratifiedKFold
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# search = RandomizedSearchCV(lgbm_model, param_distributions=param_grid, n_iter=20, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1, random_state=42)
# search.fit(x_train, y_train)

# # Print best parameters and best score
# print("Best CV score: {:.4f}".format(search.best_score_))
# print("Best parameters:", search.best_params_)

# # Get the best model
# best_model = search.best_estimator_

In [ ]:
# Build the lightgbm model based on best_model parameter from hyperparameter tuning 
lgbm_model = LGBMClassifier(max_depth= 50,
                            num_class=3, 
                            objective='multiclass',
                            preprocessor__cat__imputer__strategy='most_frequent')
lgbm_model.fit(x_train, y_train)

# make prediction
y_pred_lgbm=lgbm_model.predict(x_test)

# Model Evaluation
## Accuracies


In [ ]:
def evaluate_model(prediction):
    ''' # function for evaluation matrix '''
    # Create the confusion matrix
    cm = confusion_matrix(y_test, prediction)
    ConfusionMatrixDisplay(confusion_matrix=cm).plot()
    accuracy = round(accuracy_score(y_test, prediction), 5)
    return accuracy

In [ ]:
rf_accuracy = evaluate_model(y_pred_rf)

print('Random Forest accuracy: ',rf_accuracy)
print(classification_report(y_test, y_pred_rf))


In [ ]:
lgbm_accuracy = evaluate_model(y_pred_lgbm)

print('LightGBM accuracy: ',lgbm_accuracy)
print(classification_report(y_test, y_pred_lgbm))

## Check for overfitting
The training accuracy for Random Forest model is 0.9462 which we can say is a bit overfitting the data compared to the LightGBM model

In [ ]:
print('Random Forest Training Accuracy : {:.4f}'.format(metrics.accuracy_score(y_train, rf_model.predict(x_train))))
print('LightGBM Training Accuracy : {:.4f}'.format(metrics.accuracy_score(y_train, lgbm_model.predict(x_train))))

# Implement model on submission data


In [ ]:
df_test  = pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv')
# extraxt ids
test_ids = df_test['id']
df_test.drop(columns=['id'],inplace = True)
df_test.drop(columns_drop, axis=1, inplace=True)


In [ ]:
# make predictions
y_pred = lgbm_model.predict(df_test)

In [ ]:
# Create submission file
submission = pd.DataFrame({
    'id': test_ids,
    'Target': y_pred
})
submission

In [ ]:
# Apply the reverse mapping to the 'Target' column
submission['Target'] = submission['Target'].map({2: 'Graduate', 1: 'Enrolled', 0: 'Dropout'})
submission

In [ ]:
submission.to_csv("submission.csv", index=False)

# Conclusion
There is definitely a lot of areas for improvements especially in the hyperparameter tuning for LGBM model. <br>
I would also consider building a pipeline in future projects <br>
More explanations should be provided during EDA based on the plots as well